## Initialization
 * Import Janus Client modules
 * Setup client parameter (controller URL, user, ssh key, etc.)
 * Create a client instance
 * Create sessions for each demo job
 * Attach services to each session (host/container/profile mapping)

### Profile definition example
 * 400g-demo.yaml
 * This, along with other profiles, are maintained on the Janus controller node
  
```yaml
features:
  rdma-demo:
    devices:
      - devprefix: "/dev/infiniband"
        names:
          - rdma_cm
          - uverbs
    caps:
      - IPC_LOCK
      - NET_ADMIN
    limits:
      - Name: memlock
        Soft: -1
        Hard: -1

profiles:
  lbnl-400g-1:
    cpu: 4
    affinity: network
    serv_port_range: null
    mgmt_net: bridge
    data_net:
      name: net3001_eth200
      ipv4_addr:
        - 10.33.1.20
        - 10.33.1.21
    features:
      - rdma-demo
    privileged: false

  lbnl-400g-2:
    cpu: 4
    affinity: network
    serv_port_range: null
    mgmt_net: bridge
    data_net:
      name: net3002_eth200
      ipv4_addr:
        - 10.33.2.20
        - 10.33.2.21
    features:
      - rdma-demo
    privileged: false

  lbnl-400g-host:
    cpu: null
    mem: null
    serv_port_range: null
    mgmt_net: host
    privileged: false
    volumes:
      - data
      - tmp
      - proc
```

In [1]:
import time
from janus_client import Client, Session, Service
from ESCPeval import setup, run_job, stop_job


JANUS_URL="https://nersc-srv-1.testbed100.es.net:5050"

user = 'kissel'
mykey = 'ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQC/DAAkHubwAfjOSVG67tvwvUrESRoig+OAk5htH0mLK7QaN4gTv/C3ritJH3PkRzIJyOAJVS1ssjiBtMpI8Lb8NZCYKDZdpH/803cPpd2KqI2oAWRZQcjamKYs6SlM9r380wZ069r143ldaWPU3CJ6ye2+yx0Gifdivutnk5eQ1DxdK/VbukMzWeDdO8T1QXHxqP6lRy5PUkDKlY6Ec8i8YLYijhBoMlK0NL23d9ZDrVFKYe4D9OBp0CCKzkz9a7qgLOoYvCGRzN7BXIztJkYzziFAYn4Ru0RTyHM81V8jp4H0E2EM7BDtUUBac/sk7pJS7w2rIRZtPDsFA7qFvw+1 dtnaas@a83afe25e688'

client = Client(JANUS_URL, auth=(user, 'd3m0p4ss'))

sess_rdma1 = client.getSession()
srv1 = Service(instances=['nersc-tbn-6'], image='dtnaas/ofed:5.4-3',
               profile='lbnl-400g-1', username=user, public_key=mykey)
srv2 = Service(instances=['nersc-tbn-7'], image='dtnaas/ofed:5.4-3',
               profile='lbnl-400g-2', username=user, public_key=mykey)
sess_rdma1.addService(srv1)
sess_rdma1.addService(srv2)

sess_rdma2 = client.getSession()
srv1 = Service(instances=['nersc-srv-1'], image='dtnaas/ofed:5.4-3',
               profile='lbnl-400g-1', username=user, public_key=mykey)
srv2 = Service(instances=['nersc-dtnaas-1'], image='dtnaas/ofed:5.4-3',
               profile='lbnl-400g-2', username=user, public_key=mykey)
sess_rdma2.addService(srv1)
sess_rdma2.addService(srv2)

sess_tcp = client.getSession()
srv = Service(instances=['nersc-tbn-1', 'nersc-tbn-2'],
              image='dtnaas/tools:latest',
              profile='lbnl-400g-host',
              username=user,
              public_key=mykey)
sess_tcp.addService(srv)

print (f"{sess_rdma1}\n\n{sess_rdma2}\n\n{sess_tcp}")

id: 90bf2afa-6472-401f-820c-cd0e38707820
allocated: False
requests: [{'instances': ['nersc-tbn-6'], 'image': 'dtnaas/ofed:5.4-3', 'profile': 'lbnl-400g-1', 'kwargs': {'USER_NAME': 'kissel', 'PUBLIC_KEY': 'ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQC/DAAkHubwAfjOSVG67tvwvUrESRoig+OAk5htH0mLK7QaN4gTv/C3ritJH3PkRzIJyOAJVS1ssjiBtMpI8Lb8NZCYKDZdpH/803cPpd2KqI2oAWRZQcjamKYs6SlM9r380wZ069r143ldaWPU3CJ6ye2+yx0Gifdivutnk5eQ1DxdK/VbukMzWeDdO8T1QXHxqP6lRy5PUkDKlY6Ec8i8YLYijhBoMlK0NL23d9ZDrVFKYe4D9OBp0CCKzkz9a7qgLOoYvCGRzN7BXIztJkYzziFAYn4Ru0RTyHM81V8jp4H0E2EM7BDtUUBac/sk7pJS7w2rIRZtPDsFA7qFvw+1 dtnaas@a83afe25e688'}}, {'instances': ['nersc-tbn-7'], 'image': 'dtnaas/ofed:5.4-3', 'profile': 'lbnl-400g-2', 'kwargs': {'USER_NAME': 'kissel', 'PUBLIC_KEY': 'ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQC/DAAkHubwAfjOSVG67tvwvUrESRoig+OAk5htH0mLK7QaN4gTv/C3ritJH3PkRzIJyOAJVS1ssjiBtMpI8Lb8NZCYKDZdpH/803cPpd2KqI2oAWRZQcjamKYs6SlM9r380wZ069r143ldaWPU3CJ6ye2+yx0Gifdivutnk5eQ1DxdK/VbukMzWeDdO8T1QXHxqP6lRy5PUkDKlY6Ec8i8YL

## DTN Service Startup
 * Start each session created in previous step
 * Check status
 * View endpoint information

In [8]:
res = sess_rdma1.start()
print (sess_rdma1.status())

res = sess_rdma2.start()
print (sess_rdma2.status())

res = sess_tcp.start()
print (sess_tcp.status())

id: 46, service: nersc-tbn-6, errors: []
id: 46, service: nersc-tbn-7, errors: []

id: 47, service: nersc-srv-1, errors: []
id: 47, service: nersc-dtnaas-1, errors: []

id: 48, service: nersc-tbn-1, errors: []
id: 48, service: nersc-tbn-2, errors: []



In [9]:
print (f"-- RDMA Src Endpoints:\n{sess_rdma1.endpoints()}")
print (f"-- RDMA Dst Endpoints:\n{sess_rdma2.endpoints()}")
print (f"-- ESCP Endpoints:\n{sess_tcp.endpoints()}")

-- RDMA Src Endpoints:
nersc-tbn-6: nersc-tbn-6.testbed100.es.net:30000
nersc-tbn-7: nersc-tbn-7.testbed100.es.net:30001
-- RDMA Dst Endpoints:
nersc-srv-1: nersc-srv-1.testbed100.es.net:30000
nersc-dtnaas-1: nersc-dtnaas-1.testbed100.es.net:30001
-- ESCP Endpoints:
nersc-tbn-1: nersc-tbn-1.testbed100.es.net:30000
nersc-tbn-2: nersc-tbn-2.testbed100.es.net:30001


## Demo job setup and execution
 * SC21 module to initialize and run jobs given provided Janus sessions

In [10]:
# Perform some setup work on the remote containers
setup(sess_rdma1)
setup(sess_rdma2)
setup(sess_tcp)

[2021-11-17 13:20:09,250] INFO: Setting up environment on nersc-tbn-6
[2021-11-17 13:20:11,203] INFO: Setting up environment on nersc-tbn-7
[2021-11-17 13:20:14,934] INFO: Setting up environment on nersc-srv-1
[2021-11-17 13:20:16,306] INFO: Setting up environment on nersc-dtnaas-1
[2021-11-17 13:20:18,742] INFO: Setting up environment on nersc-tbn-1
[2021-11-17 13:20:18,742] INFO: Setting up environment on nersc-tbn-2


In [11]:
# Run RDMA job one
hndl_r1 = run_job(sess_rdma1)

Output()

In [12]:
# Run RDMA job two
hndl_r2 = run_job(sess_rdma2) 

Output()

In [13]:
# Run TCP job
hndl_t1 = run_job(sess_tcp)

Output()

In [10]:
# Stop all job threads
stop_job(sess_rdma1, hndl_r1)
stop_job(sess_rdma2, hndl_r2)
stop_job(sess_tcp, hndl_t1)

[2021-11-17 13:11:36,628] INFO: Stopped job
[2021-11-17 13:11:36,953] INFO: Stopped job
[2021-11-17 13:11:50,216] INFO: Stopped job


## Janus stop and cleanup routines
 * It is possible to stop/start containers or destroy the sessions once workflow is complete

In [5]:
sess_rdma1.stop()
sess_rdma2.stop()
sess_tcp.stop()

In [6]:
sess_rdma1.destroy()
sess_rdma2.destroy()
sess_tcp.destroy()